In [1]:
import numpy as np
from qiskit import transpile
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SPSA,SLSQP, POWELL
from qiskit.primitives import Estimator
from qiskit_algorithms.utils import algorithm_globals
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit.quantum_info import Statevector, SparsePauliOp
import scipy
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_nature.second_q.algorithms.initial_points import HFInitialPoint
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Create the all possible excitations
num_spartial_orbital = 4
num_spin_orbitals = num_spartial_orbital * 2

# Initialize the mapper
mapper = JordanWignerMapper()
#Create an identity operator
I = FermionicOp({'': 1.0}, num_spin_orbitals =num_spin_orbitals)
I = mapper.map(I)

 #list of occupied orbitals
occupied = []
for i in range(num_spin_orbitals//4):
    occupied.append(i)
    occupied.append(i+num_spin_orbitals//2)
#occupied = [0,4,1,5]
    
# Generate all possible single excitations
excitations = []
def all_excitations(num_spin_orbitals):
    for i in range(num_spin_orbitals):
        for j in range(i+1, num_spin_orbitals):
            # Prevent electrons from moving from alpha spin to beta spin and beta spin to alpha
            if i != j and ((i < num_spin_orbitals // 2 and j < num_spin_orbitals // 2) or (i >= num_spin_orbitals // 2 and j >= num_spin_orbitals // 2)):
                # Only consider excitations where the first two alpha and beta spins are filled with electrons
                if (i in occupied and j not in occupied): 
                    excitation = FermionicOp({f'+_{j} -_{i}': 1.0}, num_spin_orbitals=num_spin_orbitals)
                    excitations.append(excitation)
    
        #Generate possible double excitations
        #Double excitations all from alpha or beta orbitals
            for k in range(j+1, num_spin_orbitals):
                for l in range(k+1, num_spin_orbitals):
                    if i != j and k != l and ((i < num_spin_orbitals // 2 and j < num_spin_orbitals // 2 and k < num_spin_orbitals // 2 and l < num_spin_orbitals // 2) or (i >= num_spin_orbitals // 2 and j >= num_spin_orbitals // 2 and k >= num_spin_orbitals // 2 and l >= num_spin_orbitals // 2)):
                        # Only consider excitations where the first two alpha and beta spins are filled with electrons
                        if (i in occupied and k not in occupied and j in occupied and l not in occupied): 
                            excitation = FermionicOp({f'+_{l} +_{k} -_{i} -_{j}': 1.0}, num_spin_orbitals=num_spin_orbitals)
                            excitations.append(excitation)
  
    for i in range(num_spin_orbitals // 2):
        for j in range(num_spin_orbitals // 2, num_spin_orbitals):
            for k in range(num_spin_orbitals // 2):
                for l in range(num_spin_orbitals // 2, num_spin_orbitals):
                    if i != k and j != l and i < k and j < l:
                        # Condition to ensure one alpha and one beta excitation
                        if (i in occupied and k not in occupied and j in occupied and l not in occupied): 
                           # Create the FermionicOp and add to double_exc list
                            exc = FermionicOp({f'+_{l} +_{k} -_{i} -_{j}': 1.0}, num_spin_orbitals=num_spin_orbitals)
                            excitations.append(exc)

    return excitations
excitations = all_excitations(num_spin_orbitals)
print(len(excitations))
print(excitations)

26
[FermionicOp({'+_3 +_2 -_0 -_1': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_2 -_0': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_3 -_0': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_2 -_1': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_3 -_1': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_7 +_6 -_4 -_5': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_6 -_4': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_7 -_4': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_6 -_5': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_7 -_5': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_6 +_2 -_0 -_4': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_7 +_2 -_0 -_4': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_6 +_3 -_0 -_4': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_7 +_3 -_0 -_4': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_6 +_2 -_0 -_5': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_7 +_2 -_0 -_5': 1.0}, num_spin_orbitals=8, ), FermionicOp({'+_6 +_3 -_0 -_5': 1.0}, num_spin_orbitals=8, ), Fermio

In [ ]:
# Initialize the driver and active_problem
distances = np.linspace(2, 4, 30)
for d in distances:#for d in distances:
    driver = PySCFDriver(atom=f"N 0.0 0.0 0.0; H {d} 0.0 0.0; H -0.506 0.876 0.0; H -0.506 -0.876 0.0", basis="sto-6g")
    #driver = PySCFDriver(atom=f"H 0.0 0.0 0.0; H 0.0 0.0 {d}", basis="sto-3g")
    problem = driver.run()
    print(f"Computing for bond length: {d} Å")
    active_transformer = ActiveSpaceTransformer(num_electrons=4, num_spatial_orbitals=num_spartial_orbital)
    active_problem = active_transformer.transform(problem)

    seed = 170
    algorithm_globals.random_seed = seed

    # Initialize the mapper
    mapper = JordanWignerMapper()
            
    # Map the electronic problem to a qubit operator
    qubit_op = mapper.map(active_problem.hamiltonian.second_q_op())
            
    # Initialize the UCCSD ansatz with Hartree-Fock initial state
    ansatz = UCCSD(
        active_problem.num_spatial_orbitals,
        active_problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            active_problem.num_spatial_orbitals,
            active_problem.num_particles,
            mapper
        ),
    )

    vqe = VQE(Estimator(), ansatz, SLSQP())
    vqe.initial_point = np.zeros(ansatz.num_parameters)
    NR = active_problem.nuclear_repulsion_energy 
    #print("Nuclear repulsion energy:", NR)
    # Calculate the exact energy
    #creating a ground state eigensolver(vqe)
    nr = -49.942982    
    solver = GroundStateEigensolver(mapper, vqe)
    result = solver.solve(active_problem)
    print(f"Total ground state energy = {result.total_energies}")

    # Extract the ground state wavefunction parameters
    psi_vqe = result.raw_result.optimal_point
        
    # Create the ansatz circuit with optimized parameters
    ansatz.assign_parameters(psi_vqe, inplace=True) 

    #Create the exact 

    from qiskit_aer import AerSimulator
    simulator = AerSimulator(method='statevector')
    qc = transpile(ansatz, simulator)
    qc.save_statevector()
        
    # Execute the circuit on the simulator
    result = simulator.run(qc).result()
    statevector = result.get_statevector()

    # Initialize the matrix M
    num_excitations = len(excitations)
    M = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
    S = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
    # Compute the matrix elements
    for i in range(len(excitations) +1):
        for j in range(len(excitations)+1):
            G_i = excitations[i-1]
            G_j = excitations[j-1]
            op_i = mapper.map(G_i)
            op_j = mapper.map(G_j)
            op = op_i.adjoint()@qubit_op@op_j
            oj = qubit_op@op_j
            oi = op_i.adjoint()@qubit_op
                    
            if i == j == 0:
                M[i, j] = Statevector(statevector).expectation_value(qubit_op)
                S[i, j] = 1.0
            elif i==0 and j > 0:
                M[i, j] = Statevector(statevector).expectation_value(oj)
                S[i, j] = Statevector(statevector).expectation_value(op_j)
            elif i>0 and j==0:
                M[i, j] = Statevector(statevector).expectation_value(oi)
                S[i, j] = Statevector(statevector).expectation_value(op_i.adjoint())
            else:
                M[i, j] = Statevector(statevector).expectation_value(op)
                S[i, j] = Statevector(statevector).expectation_value(op_i.adjoint()@op_j)                      
    
    cond_num = np.linalg.cond(S)
    print("condition number:", cond_num)

    #eigval_exact, ev = scipy.linalg.eigh(M, S) 
    #print("....", eigval_exact)
    #total_energy = eigval_exact + nr
    #print(total_energy)


Computing for bond length: 2.0 Å
Total ground state energy = [-55.79504201]
condition number: 2.7284707930894148
Computing for bond length: 2.0689655172413794 Å
Total ground state energy = [-55.78810705]
condition number: 2.9566889293778074
Computing for bond length: 2.1379310344827585 Å
Total ground state energy = [-55.78238204]
condition number: 3.1982810349834194
Computing for bond length: 2.206896551724138 Å
Total ground state energy = [-55.77771495]
condition number: 3.449595057513806
Computing for bond length: 2.2758620689655173 Å
Total ground state energy = [-55.77395403]
condition number: 3.7070534935043975
Computing for bond length: 2.344827586206897 Å
Total ground state energy = [-55.77095283]
condition number: 3.96659480542528
Computing for bond length: 2.413793103448276 Å
Total ground state energy = [-55.76857602]
condition number: 4.2193262752743035
Computing for bond length: 2.4827586206896552 Å
Total ground state energy = [-55.82164255]
condition number: 122.947960821535

In [ ]:
# Extract the ground state wavefunction parameters
psi_vqe = result.raw_result.optimal_point
    
# Create the ansatz circuit with optimized parameters
ansatz.assign_parameters(psi_vqe, inplace=True) 

#Create the exact 

from qiskit_aer import AerSimulator
simulator = AerSimulator(method='statevector')
qc = transpile(ansatz, simulator)
qc.save_statevector()
    
# Execute the circuit on the simulator
result = simulator.run(qc).result()
statevector = result.get_statevector()

### No Shots

In [ ]:
# Initialize the matrix M
num_excitations = len(excitations)
M = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
S = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
# Compute the matrix elements
for i in range(len(excitations) +1):
    for j in range(len(excitations)+1):
        G_i = excitations[i-1]
        G_j = excitations[j-1]
        op_i = mapper.map(G_i)
        op_j = mapper.map(G_j)
        op = op_i.adjoint()@qubit_op@op_j
        oj = qubit_op@op_j
        oi = op_i.adjoint()@qubit_op
                
        if i == j == 0:
            M[i, j] = Statevector(statevector).expectation_value(qubit_op)
            S[i, j] = 1.0
        elif i==0 and j > 0:
            M[i, j] = Statevector(statevector).expectation_value(oj)
            S[i, j] = Statevector(statevector).expectation_value(op_j)
        elif i>0 and j==0:
            M[i, j] = Statevector(statevector).expectation_value(oi)
            S[i, j] = Statevector(statevector).expectation_value(op_i.adjoint())
        else:
            M[i, j] = Statevector(statevector).expectation_value(op)
            S[i, j] = Statevector(statevector).expectation_value(op_i.adjoint()@op_j)                      
cond_num = np.linalg.cond(S)
print("condition number:\n", cond_num)

eigval_exact, ev = scipy.linalg.eigh(M, S) 
print("....", eigval_exact)
total_energy = eigval_exact + nr
print(total_energy)


### 500 Shots

In [ ]:
from qiskit_aer.primitives import Estimator
#for shots in [10**i for i in range(3, 4)]:
estimator = Estimator(
            run_options={"shots": 500},
            transpile_options={"seed_transpiler": 42},
        approximation=True)

eigenvalues_500 = []
# Initialize the matrix M
num_excitations = len(excitations)
M = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
S = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
# Compute the matrix elements
for t in range(1):     
    for i in range(len(excitations) +1):
        for j in range(len(excitations)+1):
            G_i = excitations[i-1]
            G_j = excitations[j-1]
            op_i = mapper.map(G_i)
            op_j = mapper.map(G_j)
            op = op_i.adjoint()@qubit_op@op_j
            oj = qubit_op@op_j
            oi = op_i.adjoint()@qubit_op
                    
            if i == j == 0:
                M[i, j] =  estimator.run(ansatz, qubit_op).result().values[0]
                S[i, j] = 1
            elif i==0 and j > 0:
                M[i, j] = estimator.run(ansatz, oj).result().values[0]
                S[i, j] = estimator.run(ansatz, op_j).result().values[0]
            elif i>0 and j==0:
                M[i, j] = estimator.run(ansatz, oi).result().values[0]
                S[i, j] = estimator.run(ansatz, op_i.adjoint()).result().values[0]
            else:
                M[i, j] = estimator.run(ansatz, op).result().values[0]
                S[i, j] = estimator.run(ansatz, op_i.adjoint()@op_j).result().values[0]
    print("Matrix M:\n", S)                       
    eigval, ev = scipy.linalg.eigh(M, S) 
   
    eigval = eigval + nr
    eigenvalues_500.append(eigval)
print("\n", eigenvalues_500)

### 1k Shots

In [ ]:
from qiskit_aer.primitives import Estimator
#for shots in [10**i for i in range(3, 4)]:
estimator = Estimator(
            run_options={"shots": 1000},
            transpile_options={"seed_transpiler": 42},
        approximation=True)

eigenvalues_1k = []
# Initialize the matrix M
num_excitations = len(excitations)
M = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
S = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
# Compute the matrix elements
for t in range(1):     
    for i in range(len(excitations) +1):
        for j in range(len(excitations)+1):
            G_i = excitations[i-1]
            G_j = excitations[j-1]
            op_i = mapper.map(G_i)
            op_j = mapper.map(G_j)
            op = op_i.adjoint()@qubit_op@op_j
            oj = qubit_op@op_j
            oi = op_i.adjoint()@qubit_op
                    
            if i == j == 0:
                M[i, j] =  estimator.run(ansatz, qubit_op).result().values[0]
                S[i, j] = 1
            elif i==0 and j > 0:
                M[i, j] = estimator.run(ansatz, oj).result().values[0]
                S[i, j] = estimator.run(ansatz, op_j).result().values[0]
            elif i>0 and j==0:
                M[i, j] = estimator.run(ansatz, oi).result().values[0]
                S[i, j] = estimator.run(ansatz, op_i.adjoint()).result().values[0]
            else:
                M[i, j] = estimator.run(ansatz, op).result().values[0]
                S[i, j] = estimator.run(ansatz, op_i.adjoint()@op_j).result().values[0]
    print("Matrix M:\n", S)                       
    eigval, ev = scipy.linalg.eigh(M, S) 
   
    eigval = eigval + nr
    eigenvalues_1k.append(eigval)
print("\n", eigenvalues_1k)

### 10k Shots

In [ ]:
from qiskit_aer.primitives import Estimator
#for shots in [10**i for i in range(3, 4)]:
estimator = Estimator(
            run_options={"shots": 10000},
            transpile_options={"seed_transpiler": 42},
        approximation=True)

eigenvalues_10k = []
# Initialize the matrix M
num_excitations = len(excitations)
M = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
S = np.zeros((num_excitations +1, num_excitations +1), dtype=complex)
# Compute the matrix elements
for t in range(10):     
    for i in range(len(excitations) +1):
        for j in range(len(excitations)+1):
            G_i = excitations[i-1]
            G_j = excitations[j-1]
            op_i = mapper.map(G_i)
            op_j = mapper.map(G_j)
            op = op_i.adjoint()@qubit_op@op_j
            oj = qubit_op@op_j
            oi = op_i.adjoint()@qubit_op
                    
            if i == j == 0:
                M[i, j] =  estimator.run(ansatz, qubit_op).result().values[0]
                S[i, j] = 1
            elif i==0 and j > 0:
                M[i, j] = estimator.run(ansatz, oj).result().values[0]
                S[i, j] = estimator.run(ansatz, op_j).result().values[0]
            elif i>0 and j==0:
                M[i, j] = estimator.run(ansatz, oi).result().values[0]
                S[i, j] = estimator.run(ansatz, op_i.adjoint()).result().values[0]
            else:
                M[i, j] = estimator.run(ansatz, op).result().values[0]
                S[i, j] = estimator.run(ansatz, op_i.adjoint()@op_j).result().values[0]
                             
    eigval, ev = scipy.linalg.eigh(M, S) 
    eigval = eigval + nr
    eigenvalues_10k.append(eigval)
print("\n", eigenvalues_10k)

## Graph

In [ ]:
import matplotlib.pyplot as plt

# Exact eigenvalues
exact_gr = total_energy[0]
exact_1_ex = total_energy[1]
exact_2_ex = total_energy[2]
exact_3_ex = total_energy[3]
exact_4_ex = total_energy[4]
exact_5_ex = total_energy[5]
exact_6_ex = total_energy[6]
exact_7_ex = total_energy[7]
exact_8_ex = total_energy[8]
exact_9_ex = total_energy[9]
exact_10_ex = total_energy[10]
exact_11_ex = total_energy[11]
exact_12_ex = total_energy[12]
exact_13_ex = total_energy[13]
exact_14_ex = total_energy[14]
exact_15_ex = total_energy[15]
exact_16_ex = total_energy[16]
exact_17_ex = total_energy[17]
exact_18_ex = total_energy[18]
exact_19_ex = total_energy[19]
exact_20_ex = total_energy[20]
exact_21_ex = total_energy[21]
exact_22_ex = total_energy[22]
exact_23_ex = total_energy[23]
exact_24_ex = total_energy[24]
exact_25_ex = total_energy[25]
exact_26_ex = total_energy[26]


# Shot counts
shots = [500, 1000, 10000] 


In [ ]:
gr = [means_500[0], means_1k[0], means_10k[0]]
ex1 = [means_500[1], means_1k[1], means_10k[1]]
ex2 = [means_500[2], means_1k[2], means_10k[2]]
ex3 = [means_500[3], means_1k[3], means_10k[3]]
ex4 = [means_500[4], means_1k[4], means_10k[4]]
ex5 = [means_500[5], means_1k[5], means_10k[5]]
ex6 = [means_500[6], means_1k[6], means_10k[6]]
ex7 = [means_500[7], means_1k[7], means_10k[7]]
ex8 = [means_500[8], means_1k[8], means_10k[8]]
ex9 = [means_500[9], means_1k[9], means_10k[9]]
ex10 = [ means_500[10], means_1k[10], means_10k[10]]
ex11 = [ means_500[11], means_1k[11], means_10k[11]]
ex12 = [ means_500[12], means_1k[12], means_10k[12]]
ex13 = [ means_500[13], means_1k[13], means_10k[13]]
ex14 = [ means_500[14], means_1k[14], means_10k[14]]
ex15 = [ means_500[15], means_1k[15], means_10k[15]]
ex16 = [ means_500[16], means_1k[16], means_10k[16]]
ex17 = [ means_500[17], means_1k[17], means_10k[17]]
ex18 = [ means_500[18], means_1k[18], means_10k[18]]
ex19 = [ means_500[19], means_1k[19], means_10k[19]]
ex20 = [ means_500[20], means_1k[20], means_10k[20]]


# variance compiled
var_gr = [val_500[0], val_1k[0], val_10k[0]]
var_ex1 = [val_500[1], val_1k[1], val_10k[1]]
var_ex2 = [val_500[2], val_1k[2], val_10k[2]]
var_ex3 = [val_500[3], val_1k[3], val_10k[3]]
var_ex4 = [val_500[4], val_1k[4], val_10k[4]]
var_ex5 = [val_500[5], val_1k[5], val_10k[5]]
var_ex6 = [val_500[6], val_1k[6], val_10k[6]]
var_ex7 = [val_500[7], val_1k[7], val_10k[7]]
var_ex8 = [val_500[8], val_1k[8], val_10k[8]]
var_ex9 = [val_500[9], val_1k[9], val_10k[9]]
var_ex10 = [ val_500[10], val_1k[10], val_10k[10]]
var_ex11 = [ val_500[11], val_1k[11], val_10k[11]]
var_ex12 = [ val_500[12], val_1k[12], val_10k[12]]
var_ex13 = [ val_500[13], val_1k[13], val_10k[13]]
var_ex14 = [ val_500[14], val_1k[14], val_10k[14]]
var_ex15 = [ val_500[15], val_1k[15], val_10k[15]]
var_ex16 = [ val_500[16], val_1k[16], val_10k[16]]
var_ex17 = [ val_500[17], val_1k[17], val_10k[17]]
var_ex18 = [ val_500[18], val_1k[18], val_10k[18]]
var_ex19 = [ val_500[19], val_1k[19], val_10k[19]]
var_ex20 = [ val_500[20], val_1k[20], val_10k[20]]
var_ex21 = [ val_500[21], val_1k[21], val_10k[21]]
var_ex22 = [ val_500[22], val_1k[22], val_10k[22]]
var_ex23 = [ val_500[23], val_1k[23], val_10k[23]]
var_ex24 = [ val_500[24], val_1k[24], val_10k[24]]
var_ex25 = [ val_500[25], val_1k[25], val_10k[25]]
var_ex26 = [ val_500[26], val_1k[26], val_10k[26]]


In [ ]:
# Scale factor for error bars 
scale_factor = 3
# Create a figure
fig, ax1 = plt.subplots(figsize=(10, 5))

# Plot exact values as horizontal lines
ax1.axhline(y=exact_gr, color='r', linestyle='-')
ax1.axhline(y=exact_1_ex, color='b', linestyle='-', label='Exact energy')
ax1.axhline(y=exact_2_ex, color='y', linestyle='-')
ax1.axhline(y=exact_3_ex, color='violet', linestyle='-')
ax1.axhline(y=exact_4_ex, color='black', linestyle='-')
ax1.axhline(y=exact_5_ex, color='g', linestyle='-')
ax1.axhline(y=exact_6_ex, color='brown', linestyle='-')
ax1.axhline(y=exact_7_ex, color='b', linestyle='-')
ax1.axhline(y=exact_8_ex, color='violet', linestyle='-')
ax1.axhline(y=exact_9_ex, color='black', linestyle='-')
#ax1.axhline(y=exact_10_ex, color='black', linestyle='-')



# Plot predicted values as dots with scaled RMSD as error bars
ax1.errorbar(shots, gr, yerr=var_gr, fmt='o', color='r', capsize=5, markersize=5)
ax1.errorbar(shots, ex1, yerr=var_ex1, fmt='o', color='b', label='Measured energy', capsize=5, markersize=5)
ax1.errorbar(shots, ex2, yerr=var_ex2, fmt='o', color='y', capsize=5, markersize=5)
ax1.errorbar(shots, ex3, yerr=var_ex3, fmt='o', color='violet', capsize=5, markersize=5)
ax1.errorbar(shots, ex4, yerr=var_ex4, fmt='o', color='black', capsize=5, markersize=5)
ax1.errorbar(shots, ex5, yerr=var_ex5, fmt='o', color='g', capsize=5, markersize=5)
ax1.errorbar(shots, ex6, yerr=var_ex6, fmt='o', color='brown', capsize=5, markersize=5)
ax1.errorbar(shots, ex7, yerr=var_ex7, fmt='o', color='b', capsize=5, markersize=5)
ax1.errorbar(shots, ex8, yerr=var_ex8, fmt='o', color='violet', capsize=5, markersize=5)
ax1.errorbar(shots, ex9, yerr=var_ex9, fmt='o', color='black', capsize=5, markersize=5)
#ax1.errorbar(shots, ex10, yerr=var_ex10, fmt='o', color='black', capsize=5, markersize=5)



# Set y-axis limit
ax1.set_ylim(-2.5, -1.1)
plt.xlabel('Shot Count')
ax1.set_ylabel('Energy in hartree')
ax1.set_title('QSE H_4cond318(3.5)Deviation Between Exact Energy and Measured Energy with Variance')
ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fontsize='small', ncol=4)
#plt.grid(True)
plt.xscale('log')  
plt.show()

